In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import os
import pandas as pd
import re

In [3]:
input_data_directory = '../raw-data/'
output_data_directory = '../clean-data/'

In [5]:
location_codes = []
dfs = []

for filename in os.listdir( input_data_directory ):
    if filename[-4:] != '.txt':
        continue
    
    full_filename = f'{input_data_directory}{filename}'
    df = pd.read_csv( full_filename, sep = '\t', comment = '#' )
    df = df[df.agency_cd == 'USGS'].reset_index( drop = True )

    with open( full_filename, 'r' ) as file:
        lines = [ line for line in file.readlines() if re.match( r'^#\s{3}', line ) ]

        for line in lines:
            if re.search( r'USGS', line ):
                location_code = re.sub( r'\#\s*', '', line ).strip()

            if re.search( '\#\s+\d+\s+\d+', line ):
                data_codes = re.split( r'\s+', line )
                series_code = f'{data_codes[1]}_{data_codes[2]}'
                series_name = ' '.join( data_codes[3:] ).strip()

    location_codes.append( location_code )
    location_code_int = re.search( r'(\d+)', location_code ).group( 1 )
    data_col_name = '_'.join( series_name.split( ',' )[0].split( ' ' ) + [ location_code_int ] ).lower()

    if location_code_int == '02334500':
        continue

    df = df \
        .assign(
            location_code = location_code,
            series_code = series_code,
            series_name = series_name,
            data_col_name = data_col_name
        ) \
        .rename( columns = { series_code: data_col_name } ) \
        .assign( **{ data_col_name: lambda _df: _df[data_col_name].astype( float ) } )
    
    dfs.append( df )

final_df = None
for df in dfs:
    df = df[['datetime',df.data_col_name.iloc[0]]]
    if final_df is None:
        final_df = df

    else:
        final_df = final_df.merge( df, on = 'datetime' )

location_df = pd \
    .DataFrame( location_codes, columns = [ 'location_name' ] ) \
    .assign( location_code = lambda _df: _df.location_name.apply( lambda name: name.split( ' ' )[1] ).astype( int ) )

final_df.to_csv( f'{output_data_directory}/usgs-data.csv', index = False )
location_df.to_csv( f'{output_data_directory}/location-data.csv', index = False )

In [8]:
def html_to_generation_df ( bs, file_date ):
    file_date = [ int( p ) for p in file_date.split( '/' ) ]
    file_date = datetime( year = file_date[2], month = file_date[0], day = file_date[1] ).strftime( '%Y-%m-%d' )

    datetimes = []
    generations = []

    schedule_table = bs.find( 'table' )
    for tr in schedule_table.find_all( 'tr' ):
        times = tr.find( **{ 'class': 'field1' } )
        generation_mw = tr.find( **{ 'class': 'field2' } )

        if times is None or generation_mw is None:
            continue

        for time, generation in zip( times, generation_mw ):
            time = time.text
            generation = generation.text

            time_match = re.match( r'^(\d+).*?([ap]m)', time )
            time_hour = int( time_match.group( 1 ) )
            if time_match.group( 2 ) == 'pm':
                time_hour += 12

            if time_hour % 12 == 0:
                time_hour -= 12

            datetimes.extend( [ f'{file_date} {time_hour:02.0f}:{min}' for min in [ '00', '15', '30', '45' ] ] )
            generations.extend( [ float( generation ) ] * 4 )

    generation_df = pd.DataFrame( zip( datetimes, generations ), columns = [ 'datetime', 'generation' ] )
    return generation_df



generation_df = pd.DataFrame()
for filename in os.listdir( input_data_directory ):
    if filename[-6:] != '.mhtml':
        continue

    full_filename = f'{input_data_directory}{filename}'
    with open( full_filename, 'r' ) as file:
        file_data = re.sub( r'=3D', '=', file.read() ) # https://en.wikipedia.org/wiki/Quoted-printable
        bs = BeautifulSoup( file_data )

        for option in bs.find_all( selected = 'selected' ):
            date_match = re.search( r'(\d+\/\d+\/\d{4})', option.attrs.get( 'value', '' ) )
            if date_match:
                file_date = date_match.group( 1 )
                generation_df = pd.concat( [ generation_df, html_to_generation_df( bs, file_date ) ], axis = 0, ignore_index = True )
                break

generation_df = generation_df.sort_values( 'datetime' ).drop_duplicates()
generation_df.to_csv( f'{output_data_directory}/dam-release.csv', index = False )